### Filename we're going to play with

In [324]:
filename = "data/arielzj_2020_fvp_2stim_magnified_v3_20_08_22.csv"

In [97]:
stimuli_number = 9
response_type_list = c('during')
trial_types = c('during')
response_names = c('During')
catch_trial_number = 10 # what it's set at at the moment, arbitrary
condition_number = 3

In [98]:
trace_cutoff = 2 # mean dissimilarity for physically identical colours must be below this
antitrace_cutoff = 3.5 # mean dissimilarity accepted for maximally physically different colours must be above this
rt_cutoff = 700 # mean reaction times must be above this

In [99]:
exclude_noncompliant = FALSE

In [168]:
plotsubjects = TRUE
plotsubjects_mixed = TRUE
plot_within_between = FALSE
plotexpsummary = FALSE
across = FALSE
population = FALSE

### Initialisations

In [101]:
# live dangerously, get rid of pesky warnings
oldw <- getOption("warn")
options(warn = -1)

In [102]:
shhh <- suppressPackageStartupMessages # stops annoying warnings when loading libraries
library(tidyr)
library(plyr)
library(dplyr)
library(ggplot2)
library(gridExtra)
library(MASS)
library(Matrix)
library(reshape2)
library(ape) # stats
library(vegan) # stats
library(RColorBrewer)
library(cocor)
library(DescTools)
library(reshape2)
library(grid)
library(ggplotify)

In [103]:
# rainbowcloud theme for plotting, stolen from: 
# https://micahallen.org/2018/03/15/introducing-raincloud-plots/?utm_campaign=News&utm_medium=Community&utm_source=DataCamp.com
raincloud_theme = theme(
text = element_text(size = 10),
axis.title.x = element_text(size = 16),
axis.title.y = element_text(size = 16),
axis.text = element_text(size = 14),
axis.text.x = element_text(angle = 45, vjust = 0.5),
legend.title=element_text(size=16),
legend.text=element_text(size=16),
legend.position = "right",
plot.title = element_text(lineheight=.8, face="bold", size = 16),
panel.border = element_blank(),
panel.grid.minor = element_blank(),
panel.grid.major = element_blank(),
axis.line.x = element_line(colour = 'black', size=0.5, linetype='solid'),
axis.line.y = element_line(colour = 'black', size=0.5, linetype='solid'))

In [104]:
# stealing ability to make flat violin plots
source("https://gist.githubusercontent.com/benmarwick/2a1bb0133ff568cbe28d/raw/fb53bd97121f7f9ce947837ef1a4c65a73bffb3f/geom_flat_violin.R")

### Functions we're going to use to do the analysis later

#### Miscellaneous

In [105]:
# get plot eccentricity labels
ploteccen <- function(datadf){
    if(datadf$Circle_1[1] == -1){
            label1 = 'Foveal'
        } else {
            label1 = 'Peripheral'
        }
        if(datadf$Circle_2[1] == -1){
            label2 = 'Foveal'
        } else {
            label2 = 'Peripheral'
        }
    return(c(label1,label2))
}

e2l <- function(eccentricity){
    if(eccentricity == -1){
        return('Foveal')
    } else {
        return('Peripheral')
    }
}

l2e <- function(eccentricity){
    if(eccentricity == 'Foveal'){
        return(-1)
    } else {
        return(1)
    }
}

#### Dissimilarity Matrices

In [106]:
# obtain dissimilarity matrices with particular eccentricity values from dataframes
dissimdata <- function(datadf, colors, eccentricity1, eccentricity2){
    # select only the eccentricities we want to keep, e.g. (foveal, foveal)
    datatemp = subset(datadf, Circle_1 == eccentricity1)
    datatemp = subset(datatemp, Circle_2 == eccentricity2)
    
    # refactor the levels so they can be plotted properly later if need be
    datatemp$Color_1 <- with(datatemp, factor(Color_1, levels = colors))
    datatemp$Color_2 <- with(datatemp, factor(Color_2, levels = colors))
    
    return(datatemp)
}

In [107]:
# plot a dissimilarity matrix
dissimplot <- function(datadf,colors,dependent='color'){
    
   # labeling the plot eccentricity comparisons
    label1 <- ploteccen(datadf)[1]
    label2 <- ploteccen(datadf)[2]
    
    
    if(dependent == 'position'){ # use position to generate matrix (as a negative control)
        plot <- ggplot(datadf, aes(x = bin1, y = bin2)) +
        theme(        axis.title.x = element_blank(), axis.title.y = element_blank(),
                      plot.title = element_text(hjust = 0.5)) +
                      scale_x_discrete(limits=c(1,2,3,4,5,6,7,8,9)) + scale_y_discrete(limits=c(1,2,3,4,5,6,7,8,9))
    } else { # use the colors to generate the matrix
        plot <- ggplot(datadf, aes(x = Color_1, y = Color_2)) +
        theme(axis.text.x = element_text(colour = colors), axis.text.y = element_text(colour = colors),
                      axis.title.x = element_blank(), axis.title.y = element_blank(),
                      plot.title = element_text(hjust = 0.5))
    }
    # stuff that's standard across plot types
        plot <- plot + geom_raster(aes(fill = similarity)) +
                labs(title = paste(label1, label2, sep = " - ")) +
                scale_fill_gradientn(colours = c("white","black")) +
                guides(fill=guide_legend(title="Dissimilarity"))
    return(plot)
}

In [108]:
df2mat_asymmetry <- function(datadf,eccentricity1,eccentricity2){
    
    datadf <- dissimdata(datadf, colors, eccentricity1, eccentricity2)
    
    # aggregate over the remaining columns of interest
    nmdsdata <- aggregate(datadf, by = list(datadf$Color_1, datadf$Color_2),FUN=mean)
    nmdsdata$Color_1 <- nmdsdata$Group.1
    nmdsdata$Color_2 <- nmdsdata$Group.2

    nmdsdata = subset(nmdsdata, select = c("Color_1","Color_2","similarity"))  # get rid of unnecessary columns
    nmdsmatrix <- spread(nmdsdata, Color_1, similarity)
    # convert the dataframe to a matrix
    nmdsmatrix <- data.matrix(nmdsmatrix)
    nmdsmatrix <- nmdsmatrix[,-1] # get rid of the labels in the first column, it messes up the code
    matdf <- as.data.frame(nmdsmatrix - t(nmdsmatrix))
    matdf$colorset <- c(abcolors)
    num_colors <- length(colors)
    matdf <- matdf %>% gather(othercolor,asymmetry ,1:num_colors)
    return(matdf)
}

In [109]:
# plot an asymmetry matrix
# BROKEN - FIX AS PER YUSUKE'S CODE: https://docs.google.com/presentation/d/1TQZ2ohqZfeWYq0TATobiSZHlkJJuZZ6vqbAmOPQaUDI/edit#slide=id.p1
asymmetry_plot <- function(datadf,eccentricity1,eccentricity2,colors){
    
    # labeling the types
    label1 <- "Presented"
    label2 <- "Response Screen"
    
    datadf <- df2mat_asymmetry(datadf,eccentricity1,eccentricity2)
    
    # refactor the levels so they can be plotted properly later if need be
    datadf$colorset <- with(datadf, factor(colorset, levels = colors))
    datadf$othercolor <- with(datadf, factor(othercolor, levels = colors))
    
    plot <- ggplot(datadf, aes(x = colorset, y = othercolor)) +
        theme(axis.text.x = element_text(colour = colors), axis.text.y = element_text(colour = colors),
                      axis.title.x = element_blank(), axis.title.y = element_blank(),
                      plot.title = element_text(hjust = 0.5))
    
    # stuff that's standard across plot types
        plot <- plot + geom_raster(aes(fill = asymmetry)) +
                labs(title = paste(label1, label2, sep = " - ")) +
                scale_fill_gradientn(colours = c("blue","white","red"), limits = c(-4,4)) +
                guides(fill=guide_legend(title="Dissimilarity\nAsymmetry"))
    return(plot)
}

In [110]:
# making a matrix symmetric. Stolen from dclone: https://rdrr.io/github/datacloning/dclone/src/R/make.symmetric.R
make.symmetric <-
function(x)
{
    if (dim(x)[1] != dim(x)[2])
        stop("'x' is not sqare matrix")
    rval <- t(x)
    m <- (x[lower.tri(x)] + rval[lower.tri(rval)]) / 2
    rval[lower.tri(rval)] <- m
    rval <- t(rval)
    rval[lower.tri(rval)] <- m
    rval
}

#### Multidimensional Scaling Plots

In [111]:
# turn a dataset into a matrix
df2mat <- function(datadf,eccentricity1,eccentricity2,dependent='color'){
    
    # select only the eccentricities we want to keep, e.g. (foveal, foveal)
    datadf = subset(datadf, Circle_1 == eccentricity1 & Circle_2 == eccentricity2)
    
    if(dependent == 'position'){ # using stimulus location as a negative control
        # aggregate over the remaining columns of interest
        nmdsdata <- aggregate(datadf, by = list(datadf$bin1, datadf$bin2),FUN=mean)
        nmdsdata$bin1 <- nmdsdata$Group.1
        nmdsdata$bin2 <- nmdsdata$Group.2
        
        nmdsdata = subset(nmdsdata, select = c("bin1","bin2","similarity"))  # get rid of unnecessary columns
        nmdsmatrix <- spread(nmdsdata, bin1, similarity)
    } else { # color as per normal
        # aggregate over the remaining columns of interest
        nmdsdata <- aggregate(datadf, by = list(datadf$Color_1, datadf$Color_2),FUN=mean)
        nmdsdata$Color_1 <- nmdsdata$Group.1
        nmdsdata$Color_2 <- nmdsdata$Group.2
        
        nmdsdata = subset(nmdsdata, select = c("Color_1","Color_2","similarity"))  # get rid of unnecessary columns
        nmdsmatrix <- spread(nmdsdata, Color_1, similarity)
    }
    
    # convert the dataframe to a matrix
    nmdsmatrix <- data.matrix(nmdsmatrix)
    nmdsmatrix <- nmdsmatrix[,-1] # get rid of the labels in the first column, it messes up the code
    
    # having 0 dissimilarity between identical objects messes up the algorithm
    # changing the 0 values to an arbitrarily small value, but this is not a good long-term solution
    # need to find a better way of doing this
    nmdsmatrix[nmdsmatrix == 0] <- 0.01
    
    
    nmdsmatrix <- make.symmetric(nmdsmatrix) # averaging over Color_1 vs. Color_2 & Color_2 vs. Color_1    
    return(nmdsmatrix)
}

In [112]:
# convert a standard matrix to a distance matrix
mat2dist <- function(mat){
    
    return(distmat <- as.dist(mat))
}

Non-metric multidimensional scaling (NMDS)

In [113]:
# find the nmds fit for a dataset
nmdsfit <- function(distmatrix){
      
    fit <- isoMDS(distmatrix,k=2, maxit=100)
    return(fit)
}

In [114]:
# plot the nmds fit
nmdsplot <- function(fit, colors, abcolors, eccentricity1, eccentricity2){
    
    # get labels
    
    # change the nmdsfit back to a dataframe
    nmdsdf <- as.data.frame(fit)
    
    plot <-     ggplot(nmdsdf, aes(x = points.1, y = points.2, color=colors, fill=colors)) + geom_point(size =10) + 
                scale_colour_manual(values = abcolors) + scale_fill_manual(values = abcolors) + 
                xlab("Dimension 1") + ylab("Dimension 2") + 
                ggtitle(paste(e2l(eccentricity1),e2l(eccentricity2),'Non-metric MDS',sep=' - '))
    return(plot)
}

Classical multidimensional scaling

In [115]:
# find the cmds fit for a dataset
cmdsfit <- function(distmatrix){
    
    fit <- cmdscale(distmatrix,k=2)
    return(fit)
}

In [116]:
# plot the cmds fit
cmdsplot <- function(fit, colors, abcolors, eccentricity1, eccentricity2){
    
    # get labels
    
    # change the nmdsfit back to a dataframe
    nmdsdf <- as.data.frame(fit)
    
    plot <-     ggplot(nmdsdf, aes(x = V1, y = V2, color=colors, fill=colors)) + geom_point(size =10) + 
                scale_colour_manual(values = abcolors) + scale_fill_manual(values = abcolors) + 
                xlab("Dimension 1") + ylab("Dimension 2") + 
                ggtitle(paste(e2l(eccentricity1),e2l(eccentricity2),'Classical MDS',sep=' - '))
    return(plot)
}

#### Some control measure plots

Reaction time vs. similarity judgements

In [117]:
# similarity judgement histogram
simhistplot <- function(datadf){
    
   plot <- ggplot(datadf, aes(x = similarity)) + geom_bar(aes(y = ..prop..)) +
    scale_x_discrete(limits=c(0,1,2,3,4,5,6,7), name = 'Dissimilarity') +
    ylab('Frequency') + ylim(0,0.8)
    return(plot)
}

simhistplot_summary <- function(datadf){
    
    datadf$subject <- as.character(datadf$subject) # necessary for visualisation
    
    plot <- ggplot(datadf, aes(x = similarity)) + 
    geom_line(stat='count',aes(y = ..prop..,group = subject),color='#CC9933') +
    geom_line(stat='count',aes(y = ..prop..),size=1.5) +
    scale_x_discrete(limits=c(0,1,2,3,4,5,6,7), name = 'Dissimilarity') +
    ylab('Frequency') + ylim(0,0.8)
    return(plot)
    
}

In [118]:
# reaction time for each similarity
rsplot <- function(datadf){
    
    plot <- ggplot(datadf, aes(x= similarity, y=response_time)) + 
    stat_summary(fun.y = mean, geom = "bar") + 
    stat_summary(fun.data = mean_se, geom = "errorbar", size =0.5, aes(width=0.5)) +
    scale_x_discrete(limits=c(0,1,2,3,4,5,6,7), name = 'Dissimilarity') + ylab('Reaction Time (ms)') +
    theme(legend.position = "none") +
    ylim(0,4000) # anyone taking more than 4 seconds has probably mindwandered
    
    return(plot)
}

rsplot_summary <- function(datadf){
    
    datadf$subject <- as.character(datadf$subject) # necessary for visualisation
    
    
    
    plot <- ggplot(datadf, aes(x= similarity, y=response_time,group = subject, color = subject)) + 
    stat_summary(fun.y = mean, geom = "line", size=0.8) + 
    #stat_summary(fun.data = mean_se, geom = "errorbar", size =0.5, aes(width=0.5)) +
    scale_x_discrete(limits=c(0,1,2,3,4,5,6,7), name = 'Dissimilarity') + ylab('Mean Reaction Time (ms)') +
    theme(legend.position = "none") +
    ylim(0,4000) # anyone taking more than 4 seconds has probably mindwandered 
    
    return(plot)
    
}

In [119]:
# reaction time raincloud plot
rsplot_raincloud <- function(datadf,xtype='linear'){
    
    datadf$subject <- as.character(datadf$subject) # necessary for visualisation  
    datadf$similarity <- as.character(datadf$similarity) # necessary for visualisation
    
    ylabtext = 'Reaction Time (ms)'
    
    plot <- ggplot(datadf, aes(y = response_time, x = similarity, fill = similarity)) +
            geom_flat_violin(position = position_nudge(x = .2, y = 0), alpha = .8) +
            geom_point(aes(y = response_time, color = similarity),
                   position = position_jitter(width = .15), size = .5, alpha = 0.8) +
            geom_boxplot(width = .1, guides = FALSE, outlier.shape = NA, alpha = 0.5) +
            expand_limits(x = 5.25) +
            guides(fill = FALSE) +
            guides(color = FALSE) +
            scale_color_brewer(palette = "Spectral") +
            scale_fill_brewer(palette = "Spectral") +
            xlab('Dissimilarity') + ylab("Reaction Time (ms)")
            # coord_flip() +
            theme_bw() +
            raincloud_theme
    
    if(xtype == 'log'){
        plot <- plot + scale_y_continuous(trans='log10') + ylim(0,15000)
    } else{
        plot <- plot + ylim(0,5000)
    }
    
    return(plot)
}

In [120]:
# correlation between reaction times and similarity judgements
# grouping at individual trial, individual participant, experiment or entire population level
rt_similarity_cor <- function(datadf,level='participant'){
        
    if(level=="participant"){
        datadf <- datadf %>% 
                group_by(subject) %>% 
                mutate(rt_similarity_correlation = cor(similarity,response_time))
        datadf <- aggregate(datadf, by=list(datadf$subject), FUN = mean)

                
    }
    return(datadf)
    
}

rt_similarity_plot <- function(datadf,xlabel='BLANK'){
    
    datadf <- rt_similarity_cor(datadf)
    
    datadf[xlabel] = xlabel
    
    plot <- ggplot(datadf,aes(x=xlabel,y=rt_similarity_correlation)) + 
                geom_boxplot() + 
                geom_dotplot(binaxis='y',stackdir='center',dotsize=0.75) +
                theme(text = element_text(size=15)) + xlab("")
                ggtitle(title)
    
    plot <- plot + ylab("Correlation (Spearman)") + ylim(-1,1)
    plot <- plot + geom_hline(yintercept=0, linetype="dashed", color = "blue")
    return(plot)
}

In [121]:
# mean reaction time vs. antitrace-trace difference
rt_difference <- function(datadf){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    for(ID in subjectlist){
        tempdf <- subset(datadf, subject == ID)
        datadf$difference[datadf$subject == ID] = unlist(antitrace_avg(tempdf),use.names=FALSE)[1] - 
                   unlist(trace_avg(tempdf),use.names=FALSE)[1]
    }
        datadf <- aggregate(datadf, by=list(datadf$subject), FUN = mean)
    return(datadf)
}

rt_difference_plot <- function(datadf,xtype='linear'){
    
    datadf$subject <- as.character(datadf$subject) # necessary for visualisation
    
    plot <- ggplot(rt_difference(datadf), aes(x=response_time, y=difference)) +
            geom_point() + xlab("Mean Reaction Time") + ylab("Antitrace - Trace")
    
    if(xtype == 'log'){
        plot <- plot + scale_x_continuous(trans='log10')
    } else{
        plot <- plot + xlim(0,5000)
    }
    plot <- plot + geom_smooth(method=lm) # linear line of best fit
    return(plot)
}

In [122]:
# mean reaction time vs. catch trial score
rt_catch <- function(datadf){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    for(ID in subjectlist){
        tempdf <- subset(datadf, subject == ID)
        datadf$catch_score[datadf$subject == ID] = unlist(catch_score(tempdf),use.names=FALSE)[1]
    }
        datadf <- aggregate(datadf, by=list(datadf$subject), FUN = mean)
    return(datadf)
}

rt_catch_plot <- function(datadf,xtype='linear',label=FALSE){
    
    datadf <- rt_catch(datadf)
    datadf$subject <- as.character(datadf$subject) # necessary for visualisation
    
    plot <- ggplot(datadf, aes(x=response_time, y=catch_score)) +
            geom_point() + xlab("Mean Reaction Time") + ylab("Catch Score")
    
    if(xtype == 'log'){
        plot <- plot + scale_x_continuous(trans='log10')
    } else{
        plot <- plot + xlim(0,5000)
    }
    plot <- plot + geom_smooth(method=lm) + ylim(0,1) # linear line of best fit
    #plot <- plot + geom_point(aes(color = subject))
    if(label){
        plot <- plot + geom_text(aes(label = subject),check_overlap=TRUE)
    }
    return(plot)
}

In [123]:
# mean antitrace-trace difference vs. catch trial score
catch_difference <- function(datadf){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    for(ID in subjectlist){
        tempdf1 <- subset(datadf, subject == ID & trial_type!= 'catch') # remove catch trials for difference calculation
        tempdf2 <- subset(datadf, subject == ID) # keep catch trials for score calculation
        datadf$difference[datadf$subject == ID] = 
                   unlist(antitrace_avg(tempdf1),use.names=FALSE)[1] - 
                   unlist(trace_avg(tempdf1),use.names=FALSE)[1]
        datadf$catch_score[datadf$subject == ID] = unlist(catch_score(tempdf2),use.names=FALSE)[1]
    }
        datadf <- aggregate(datadf, by=list(datadf$subject), FUN = mean)
    return(datadf)
}

catch_difference_plot <- function(datadf,xtype='linear',label=FALSE){
    
    datadf <- catch_difference(datadf)
    datadf$subject <- as.character(datadf$subject) # necessary for visualisation
    
    plot <- ggplot(datadf, aes(x=difference, y=catch_score)) +
            geom_point() + xlab("Antitrace - Trace") + ylab("Catch Score")
    
    if(xtype == 'log'){
        plot <- plot + scale_x_continuous(trans='log10')
    } else{
        plot <- plot
    }
    plot <- plot + geom_smooth(method=lm) + ylim(0,1) # linear line of best fit
    #plot <- plot + geom_point(aes(color = subject))
    if(label){
        plot <- plot + geom_text(aes(label = subject),check_overlap=TRUE)
    }
    return(plot)
}

In [124]:
# trace values vs. trial #
trace_by_trial_plot <- function(datadf,graph='point',mode='relative'){
    
     datadf$subject <- as.character(datadf$subject) # necessary for visualisation  
    
    datadf <- subset(datadf, Color_1 == Color_2)
    
    if(mode=='relative'){
        datadf <- datadf %>% group_by(subject) %>% mutate(trialnum = row_number())
    } else{
        datadf$trialnum <- datadf$trial_number
    }
        
    if(graph=='point'){
        plot <- ggplot(datadf, aes(x=trialnum, y = similarity, group = subject, color = subject)) + geom_point()
    } else if (graph == 'boxplot'){
        plot <- ggplot(datadf, aes(x=trialnum,y = similarity, group = trialnum)) + geom_boxplot() #????
    } else if (graph == 'lineplot'){
        plot <- ggplot(datadf, aes(x=trialnum, y = similarity,group=subject, color = subject)) + geom_line()
    }
    
    plot <- plot + ylab('Dissimilarity') +  theme(legend.position='none')
    
    return(plot)
    
}

In [125]:
# subject info
sumplot <- function(datadf){
    
    # change ms to s, add the delay for each trial
    datadf$response_time <- ((datadf$response_time + 0.125*nrow(datadf)) / 1000)
    
    plot <- ggplot(datadf, aes(x=subject, y = response_time)) + 
    stat_summary(fun.y = sum, geom = "bar") + ylim(0,1000) +
    ylab('Response Time Total') + theme(axis.title.x=element_blank(), axis.text.x = element_text(size=20))
    
    return(plot)
}

#### Check their screen size & viewing distance

In [370]:
# screen size in mm
screen_size <- function(datadf){
    
    datadf <- subset(datadf, trialcode == 'centermouse') # make sure calibrations have finished first
    
    width <- as.numeric(substr(as.character(datadf$screen_size_x)[1],1,6))
    height <- as.numeric(substr(as.character(datadf$screen_size_y)[1],1,6))
    
    # use pythagoras to just get the hypotenuse. Subjects have a fixed 16/9 aspect ratio so these are all comparable
    return(sqrt(width*width + height*height))
}

In [372]:
# viewing distance in mm
view_distance <- function(datadf){
    return(as.numeric(substr(as.character(datadf$viewer_distance)[1],1,6)))
}

### Statistics

Useful functions

In [126]:
# compare the foveal and peripheral matrices to each other
fvp_test <- function(datadf,dependent='color'){
    
    if(dependent=='position'){
        # first, turn the dataframe into the relevant matrices
        fmatrix <- df2mat(datadf, l2e('Foveal'), l2e('Foveal'),'position')
        #fpmatrix <- distmatrix(datadf, l2e('Foveal'), l2e('Peripheral'))
        pmatrix <- df2mat(datadf, l2e('Peripheral'), l2e('Peripheral'),'position')
    } else{
        # first, turn the dataframe into the relevant matrices
        fmatrix <- df2mat(datadf, l2e('Foveal'), l2e('Foveal'))
        #fpmatrix <- distmatrix(datadf, l2e('Foveal'), l2e('Peripheral'))
        pmatrix <- df2mat(datadf, l2e('Peripheral'), l2e('Peripheral'))
    }
    #Fovea vs Periphery
    test <- mantel(fmatrix, pmatrix, method="spearman",permutations = 9999)
    return(test)
}

In [127]:
# compare the each of the foveal and peripheral matrices to a random uniform distribution using a ... test
# not actually sure this is conceptually possible or even makes sense
rand_test <- function(datadf){
    
    # first, turn the dataframe into the relevant matrices
    fmatrix <- df2mat(datadf, l2e('Foveal'), l2e('Foveal'))
    #fpmatrix <- distmatrix(datadf, l2e('Foveal'), l2e('Peripheral'))
    pmatrix <- df2mat(datadf, l2e('Peripheral'), l2e('Peripheral'))
    
    
    # compare
    ftest <- 
    ptest <- 
    return(list("ftest" = ftest, "ptest" = ptest)) # use index of result to get the foveal or peripheral comparison
}



In [128]:
# calculate trace average, small values presumably indicate subjects not just saying everything is completely dissimilar
# will also need a complementary method to deal with them saying everything is the same? Might be an inappropriate
# a-priori imposition though, I probably can't tell people how similar different colours should be...

trace_avg <- function(datadf){
    
    # first, turn the dataframe into the relevant matrices
    fmatrix <- df2mat(datadf, l2e('Foveal'), l2e('Foveal'))
    #fpmatrix <- distmatrix(datadf, l2e('Foveal'), l2e('Peripheral'))
    pmatrix <- df2mat(datadf, l2e('Peripheral'), l2e('Peripheral'))
    
    
    
    return(list( (sum(diag(fmatrix))/nrow(fmatrix)), (sum(diag(pmatrix))/nrow(pmatrix))))
}

In [129]:
# find the 'anti-trace' of a matrix
antitrace <- function(mat){
    
    sum = 0
    for(i in (1:nrow(mat))){
        k = as.numeric(mat[ i, as.integer((ncol(mat)/2+i) %% ncol(mat))+1 ])
        sum = sum + k
    }
    return(sum/nrow(mat))
}

In [130]:
# calculates the average of the 'anti-trace', the values I'd expect to be the most dissimilar.
# this may not be an appropriate imposition on subjects...

antitrace_avg <- function(datadf){
    
    # first, turn the dataframe into the relevant matrices
    fmatrix <- df2mat(datadf, l2e('Foveal'), l2e('Foveal'))
    #fpmatrix <- distmatrix(datadf, l2e('Foveal'), l2e('Peripheral'))
    pmatrix <- df2mat(datadf, l2e('Peripheral'), l2e('Peripheral'))
    
    return(list(antitrace(fmatrix), antitrace(pmatrix)))
}

In [131]:
# just get a simple measure of if the matrices are correlated, irrespective of treating them as distance matrices
matrixcor <- function(datadf,dependent='color'){
    
    if(dependent == 'position'){
        # first, turn the dataframe into the relevant matrices
        fmatrix <- df2mat(datadf, l2e('Foveal'), l2e('Foveal'),'position')
        #fpmatrix <- distmatrix(datadf, l2e('Foveal'), l2e('Peripheral'))
        pmatrix <- df2mat(datadf, l2e('Peripheral'), l2e('Peripheral'),'position')
    } else {
        # first, turn the dataframe into the relevant matrices
        fmatrix <- df2mat(datadf, l2e('Foveal'), l2e('Foveal'))
        #fpmatrix <- distmatrix(datadf, l2e('Foveal'), l2e('Peripheral'))
        pmatrix <- df2mat(datadf, l2e('Peripheral'), l2e('Peripheral'))
    }
    
    return(cor(c(fmatrix), c(pmatrix), method = "spearman"))
}

In [132]:
# get median reaction time
rt_avg <- function(datadf){
    return(median(datadf$response_time))
}

Run the analysis

In [133]:
# stats for all the subjects
allsubjectstats <- function(datadf){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects


    for (subjectid in subjectlist){
        
        subjectdf = subset(datadf, subject == subjectid) # select the df for subject of interest
        cat("SUBJECT: ",subjectid,'\n') # print subject id
        print("Mantel Test: ")
        print(fvp_test(subjectdf))
        print("Matrices correlation: ")
        print(matrixcor(subjectdf))
        print("Quality control measures:")
        print("Trace average:")
        print(trace_avg(subjectdf))
        print("Anti-trace average:")
        print(antitrace_avg(subjectdf))
    }
}

In [134]:
# function to aggregate everyone's data together
aggregate_df <- function(datadf,dependent='color'){

    # aggregate everyone's data together for the matrices
    everyonedata <- aggregate(datadf, by=list(
        datadf$Color_1,
        datadf$Color_2,
        datadf$Circle_1,
        datadf$Circle_2,
        datadf$bin1,
        datadf$bin2
        ), FUN=mean, 
    )

    # correct the column names
    everyonedata$Color_1 <- everyonedata$Group.1
    everyonedata$Color_2 <- everyonedata$Group.2
    everyonedata$Circle_1 <- everyonedata$Group.3
    everyonedata$Circle_2 <- everyonedata$Group.4
    everyonedata$bin1 <- everyonedata$Group.5
    everyonedata$bin2 <- everyonedata$Group.6
    
    return(everyonedata)
}

In [135]:
# compliance checker
compliance_checker <- function(datadf,eccentricity,trace_cutoff,antitrace_cutoff){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    n_subject = length(subjectlist)
    trace_fail = 0
    antitrace_fail = 0
    
    if(eccentricity == 'Foveal'){
        e_ind = 1
    } else if (eccentricity == 'Peripheral'){
        e_ind = 2
    }

    for (subjectid in subjectlist){
        subjectdf = subset(datadf, subject == subjectid) # select the df for subject of interest
        if(trace_avg(subjectdf)[e_ind] > trace_cutoff){
            trace_fail = trace_fail + 1
        }
        if(antitrace_avg(subjectdf)[e_ind] < antitrace_cutoff){
            antitrace_fail = antitrace_fail + 1
        }
    }
    print(paste("Trace non-compliance: ",trace_fail,"/",n_subject))
    print(paste("Anti-trace non-compliance: ",antitrace_fail,"/",n_subject))
}

In [136]:
# summary stats
summarystats <- function(datadf,dependent='color'){
    if(dependent=='position'){
        print("Mantel Test: ")
        print(fvp_test(aggregate_df(datadf),'position'))
        print("Matrices correlation: ")
        print(matrixcor(aggregate_df(datadf),'position'))
    } else {
        print("Mantel Test: ")
        print(fvp_test(aggregate_df(datadf)))
        print("Matrices correlation: ")
        print(matrixcor(aggregate_df(datadf)))
    }
    
    
    print("Foveal compliance:")
    compliance_checker(datadf,'Foveal')
    print("Peripheral compliance:")
    compliance_checker(datadf,'Peripheral')
}

## Actual Data Analysis Time

#### Initial data pre-processing, before getting to the subject level

In [392]:
datadf = read.csv(filename)
savestr <- substr(filename,1,nchar(filename)-4) # for saving related files later


In [393]:
# remove practice trial data
datadf <- subset(datadf, trial_number != 0)

In [394]:
# changing color values from RGB to hex for graphing purpose
datadf$Color_1 <- as.character(datadf$Color_1)
datadf$Color_1 <- revalue(datadf$Color_1, 
                                                    c(  "1" = '#FF0000',
                                                        "2" = '#FFAA00',
                                                        "3" = '#AAFF00',
                                                        "4" = '#00FF00',
                                                        "5" = '#00FFA9',
                                                        "6" = '#00A9FF',
                                                        "7" = '#0000FF',
                                                        "8" = '#AA00FF',
                                                        "9" = '#FF00AA'))
datadf$Color_2 <- as.character(datadf$Color_2)
datadf$Color_2 <- revalue(datadf$Color_2, 
                                                    c(  "1" = '#FF0000',
                                                        "2" = '#FFAA00',
                                                        "3" = '#AAFF00',
                                                        "4" = '#00FF00',
                                                        "5" = '#00FFA9',
                                                        "6" = '#00A9FF',
                                                        "7" = '#0000FF',
                                                        "8" = '#AA00FF',
                                                        "9" = '#FF00AA'))

# colors for the labels
# changed for this mixed version
colors <- c('#FF0000','#FFAA00','#AAFF00','#00FF00','#00FFA9',"#00A9FF",'#0000FF','#AA00FF',"#FF00AA")

abcolors <- sort(colors)

In [395]:
# changing from int indicators in the .csv file to more readable labels for eccentricity
foveal = -1
peripheral = 1

In [396]:
# set the maximum and minimum dissimilarity values for later analysis
min_val = 0
max_val = 6

In [397]:
# binning the positions the stimuli were displayed in to use as a control measure later
datadf$bin1 <- as.integer((datadf$Position_1+180) / 40) + 1
datadf$bin2 <- as.integer((datadf$Position_2+180) / 40) + 1

In [398]:
# adding a trial type label
label_eccen_type <- function(datadf){
    datadf <- datadf %>%
    mutate(eccen_type = case_when(Circle_1 == l2e('Foveal') & Circle_2 == l2e('Foveal') ~ 'FF',
                             Circle_1 == l2e('Foveal') & Circle_2 == l2e('Peripheral') ~ 'FP',
                             Circle_1 == l2e('Peripheral') & Circle_2 == l2e('Peripheral') ~ 'PP'))
    return(datadf)
}

In [399]:
datadf <- label_eccen_type(datadf)

Some actual data cleaning

In [400]:
# function to throw out  participants who didn't finish
remove_incomplete <- function(datadf,stimuli_number){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    for (ID in subjectlist){ # go through subject by subject
        
        trialnumber = stimuli_number*stimuli_number*condition_number + catch_trial_number
        
        if ( nrow(subset(datadf, subject == ID)) != trialnumber){ # see if they have the appropriate trial numbers
            print(paste("Subject",ID,"did not complete the task",sep=' '))
            datadf <- subset(datadf, subject != ID) # dump the subject
        }
    }
    return(datadf)
}

In [401]:
# actually throw out the subjects who didn't complete
datadf <- remove_incomplete(datadf,stimuli_number)

[1] "Subject 197308657 did not complete the task"
[1] "Subject 558344777 did not complete the task"
[1] "Subject 634992811 did not complete the task"
[1] "Subject 728255502 did not complete the task"
[1] "Subject 910252650 did not complete the task"


In [402]:
# function to label participants who meet statistical cleaning assumptions
label_noncompliant <- function(datadf,eccentricity,
                               trace_cutoff=2,antitrace_cutoff=4,rt_cutoff=1000){
    
    if(eccentricity == 'Foveal'){
        e_ind = 1
    } else if (eccentricity == 'Peripheral'){
        e_ind = 2
    }
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    for (ID in subjectlist){ # go through subject by subject
        subjectdf = subset(datadf, subject == ID) # select the ID for subject of interest 
        if(trace_avg(subjectdf)[e_ind] < trace_cutoff){
            if(antitrace_avg(subjectdf)[e_ind] > antitrace_cutoff){
                if(rt_avg(subjectdf) > rt_cutoff){
                    datadf$noncompliant[datadf$subject == ID] = 0
                } else {
                    print(paste("Subject",ID,"median reaction time of:",rt_avg(subjectdf),"ms",sep=' '))
                    datadf$noncompliant[datadf$subject == ID] = 1
                }
            } else {
                print(paste("Subject",ID,"had an antitrace of:",antitrace_avg(subjectdf)[e_ind],sep=' '))
                datadf$noncompliant[datadf$subject == ID] = 2
            }
        } else {
            print(paste("Subject",ID,"had a trace of:",trace_avg(subjectdf)[e_ind],sep=' '))
            datadf$noncompliant[datadf$subject == ID] = 3
        }
    }
    print(paste(length(unique(datadf$subject[datadf$noncompliant==0])),"of",length(unique(datadf$subject)),
                "were compliant",sep=' '))
    return(datadf)
}

In [403]:
datadf <- label_noncompliant(datadf,'Foveal')

[1] "Subject 158161839 had a trace of: 6.22222222222222"
[1] "Subject 162264632 had a trace of: 4.22222222222222"
[1] "Subject 235749592 median reaction time of: 679 ms"
[1] "Subject 765850091 median reaction time of: 627 ms"
[1] "Subject 788516555 had a trace of: 2.55555555555556"
[1] "Subject 854808809 had a trace of: 2.88888888888889"
[1] "Subject 867419577 had a trace of: 3.88888888888889"
[1] "5 of 12 were compliant"


## Statistical Analysis

#### Determine within-subject central and peripheral QDM correlation & significance

In [231]:
# return a list of the fovea matrix vs periphery matrix correlations for each subject

fvp_list <- function(datadf){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    correlation_list <- vector() # array to store the values in
    
    for (ID in subjectlist){ # go through subject by subject
        subjectdf = subset(datadf, subject == ID) # select the ID for subject of interest
        correlation_list <- c(correlation_list, (matrixcor(subjectdf)))
    }
    return(correlation_list)
}

In [232]:
# shuffle the rows of a matrix
shuffle_matrix <- function(matrix){
    shuffled <- matrix[sample(nrow(matrix)),]
    return(shuffled)
}

In [233]:
# return a list of the fovea matrix vs. shuffled periphery matrix for each subject. Pick the # shuffles, default 1000.
# returns Fisher-transformed correlations

fvp_shuffled_list <- function(datadf, shuffle_num=1000){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    
    correlation_list <- vector() # array to store the values in
    
    for (ID in subjectlist){ # go through subject by subject
        subjectdf = subset(datadf, subject == ID) # select the ID for subject of interest
        
        fmatrix <- df2mat(datadf, l2e('Foveal'), l2e('Foveal'))
        pmatrix <- df2mat(datadf, l2e('Peripheral'), l2e('Peripheral'))
        
        temp_list <- vector()
        for (i in 1:shuffle_num){ #shuffle the peripheral matrix, add values to temp list
            pmatrix_shuffled <- shufflematrix(pmatrix)
            temp_list <- c(temp_list, FisherZ(cor(c(fmatrix), c(pmatrix_shuffled), method = "spearman")))
        }
        
        correlation_list <- c(correlation_list,mean(temp_list)) # add the average correlation to the list
        }
    return(correlation_list)
}

In [234]:
# perform Mann-Whitney U test to compare the within-subject central and peripheral QDMs to randomly shuffled ones
within_subject_test <- function(original_correlations,shuffled_correlations){
    
    wilcox.test(original_correlations,shuffled_correlations)
}

In [235]:
# do the analysis

#### Determine between-subject central to central QDM correlation & significance

In [236]:
# obtain the subject similarity matrix
i_j_list <- function(datadf, eccentricity1=l2e('Foveal'), eccentricity2=l2e('Foveal')){
    
    datadf = subset(datadf, Circle_1 == eccentricity1)
    datadf = subset(datadf, Circle_2 == eccentricity2)
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    correlation_list <- vector() # array to store the values in
    
    for (ID_i in subjectlist){ 
        for (ID_j in subjectlist){
            if(ID_i != ID_j){
                subjectdf_i = subset(datadf, subject == ID_i)
                subjectdf_j = subset(datadf, subject == ID_j)

                fmatrix_i <- df2mat(subjectdf_i, l2e('Foveal'), l2e('Foveal')) # get their fovea dissimilarity matrix
                fmatrix_j <- df2mat(subjectdf_j, l2e('Foveal'), l2e('Foveal')) # get their fovea dissimilarity matrix

                correlation_list <- c(correlation_list, cor(c(fmatrix_i), c(fmatrix_j), method = "spearman"))
                }
        }
    }
    return(correlation_list)
}

In [237]:
# obtain a shuffled subject similarity matrix

In [238]:
# test where the correlation of the original SSM sits in comparison to the shuffled SSMs

In [239]:
# do the analysis

#### Determine whether within-subject central-peripheral QDM correlation is equal to or greater than between-subject central-central QDMs

In [240]:
# blah

In [241]:
# do the analysis

In [242]:
# correlation plot
corr_plot <- function(correlation_list,title='BLANK',xlabel='BLANK',f2z=FALSE){
    df <- data.frame(correlation_list) # convert list to data frame
    df[xlabel] = xlabel
    
    plot <- ggplot(df,aes(x=xlabel,y=correlation_list)) + 
                geom_boxplot() + 
                geom_dotplot(binaxis='y',stackdir='center',dotsize=0.75) +
                theme(text = element_text(size=15)) + xlab("")
                ggtitle(title)
    if(f2z){
                plot <- plot + ylab("Fisher-Transformed Correlation (Spearman)") + ylim(-1,2)
    } else {
                plot <- plot + ylab("Correlation (Spearman)") + ylim(-0.5,1)

    }
    plot <- plot + geom_hline(yintercept=0, linetype="dashed", color = "blue")
    return(plot)
}

In [243]:
within_between_plots <- function(datadf,savestr){
    
    fvp <- fvp_list(datadf)
    i_j <- i_j_list(datadf)
    
    within_plot <- corr_plot(fvp,'Within-subject FvP','Within-subject FvP')
    between_plot <- corr_plot(i_j,'Between-subject FvF','Between-subject FvF')
    within_plot_fisher <- corr_plot(FisherZ(fvp),'Within-subject FvP','Within-subject FvP',TRUE)
    between_plot_fisher <- corr_plot(FisherZ(i_j),'Between-subject FvF','Between-subject FvF',TRUE)

    # save the plots
    title <- textGrob(substr(savestr,nchar('data/')+1,nchar(savestr)))
    g <- arrangeGrob(within_plot,between_plot,
                     within_plot_fisher,between_plot_fisher,
                        nrow = 2, ncol= 2,
                    top = title)

        # save it
    ggsave(paste(savestr,"_Subject_Plots/","correlation plot comparisons_","plot.png",sep=''),g, height = 11, width = 10)
}

In [244]:
# plot within-subject FvP correlations against between-subject FvF correlations
if(plot_within_between){within_between_plots(datadf,savestr)}

Additional experimental summary graphs

In [245]:
# obtains a dataframe of traces for subjects
trace_obtainer <- function(datadf,type="trace"){
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    tracedf <- data.frame(subject=character(),ftrace=double(),ptrace=double()) # initialise an output df for the traces
    for (ID in subjectlist){
        tempdf <- subset(datadf, subject == ID)
        if(type == 'antitrace'){
            subject_trace <- antitrace_avg(tempdf)
        } else if (type == 'difference'){
            subject_trace <- unlist(antitrace_avg(tempdf),use.names=FALSE) - unlist(trace_avg(tempdf),use.names=FALSE)
        }else{
            subject_trace <- trace_avg(tempdf)
        }
        row_df <- data.frame(ID, as.numeric(subject_trace[1]),as.numeric(subject_trace[2]))
        names(row_df) <- c("subject","Fovea x Fovea","Periphery x Periphery")
        tracedf<- rbind(tracedf,row_df)
    }
    return(tracedf)
}

In [246]:
# plot the traces for the subjects individually
trace_plot <- function(datadf,type='trace',grouped=TRUE){
    
    datadf$subject <- as.character(datadf$subject) # necessary for visualisation
    
    if(type == 'antitrace'){
        trace_df <- trace_obtainer(datadf,'antitrace')
    } else if (type == 'difference'){
        trace_df <- trace_obtainer(datadf,'difference')
    } else {
        trace_df <- trace_obtainer(datadf)
    }
    
    
    tracedf <- gather(trace_df, key = "eccentricity",value = "trace", 2:3)
    
    if(grouped){
        plot <- ggplot(tracedf, aes(x = eccentricity, y=trace)) +
        geom_boxplot() + geom_dotplot(binaxis='y',stackdir='center',dotsize=0.5) +
        theme(legend.position = "none")
    } else {
        plot <- ggplot(tracedf, aes(x = eccentricity, y=trace, group = subject, color = subject)) +
        geom_line() + theme(legend.position = "none")
    }
    
    if(type == 'antitrace'){
        plot <- plot + ylim(0,7) + ylab("Mean Physically Maximally Distinct Stimulus Similarity")
    } else if (type == 'difference'){
        plot <- plot + ylim(-6,6) + ylab("Distinct - Identical")
    } else {
        plot <- plot + ylim(0,7) + ylab("Mean Physically Identical Stimulus Similarity")
    }
    
    return(plot)
}

### Plot summary of all subjects

#### Individual subjects

In [283]:
# plot summary data for all the subjects individually
allsubjectsplot <- function(datadf,savestr){
    
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects

    dir.create(paste(savestr,"Subject_Plots",sep='_')) # make a new folder to store the subject plots in

    for (subjectid in subjectlist){
        print(paste("Processing subject:",subjectid,sep=' '))
        subjectplot(datadf, subjectid,savestr)
    }
}

In [426]:
# make a plot of the key summary data for a subject - single trial type version
subjectplot <- function(datadf, subjectid,savestr){
    
    datadf = subset(datadf, subject == subjectid) # select the ID for subject of interest
    for (trial_condition in trial_types){
            subjectdf = subset(datadf, trial_type == trial_condition) # just pick one trial type at a time
    
    
        if(subjectdf$trial_type[1] != 'after_1'){
            FP = TRUE
        } else {
            FP = FALSE
        }

        # FF Matrix plot
        FFmatrix <- dissimplot(dissimdata(subjectdf, colors, l2e('Foveal'), l2e('Foveal')),colors)



        # PP Matrix plot
        PPmatrix <- dissimplot(dissimdata(subjectdf, colors, l2e('Peripheral'), l2e('Peripheral')),colors)

        # FF CMDS plot
        FFcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(subjectdf,l2e('Foveal'), l2e('Foveal')))), 
                            abcolors, abcolors, l2e('Foveal'), l2e('Foveal'))



        # PP CMDS plot
        PPcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(subjectdf,l2e('Peripheral'), l2e('Peripheral')))), 
                            abcolors, abcolors, l2e('Peripheral'), l2e('Peripheral'))

        # FF nMDS plot
        FFnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(subjectdf,l2e('Foveal'), l2e('Foveal')))), 
                            abcolors, abcolors, l2e('Foveal'), l2e('Foveal'))



        # PP nMDS plot
        PPnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(subjectdf,l2e('Peripheral'), l2e('Peripheral')))), 
                            abcolors, abcolors, l2e('Peripheral'), l2e('Peripheral'))

        # Similarity judgements
        simplot <- simhistplot(subjectdf)
        # Reaction times
        reactionplot <- rsplot(subjectdf)
        # summary plot
        rainplot <- rsplot_raincloud(subjectdf,'log')

        # trace plots
        traceplot <- trace_plot(subjectdf)
        antiplot <- trace_plot(subjectdf,'antitrace')
        differenceplot <- trace_plot(subjectdf,'difference')

        # asymmetry plot
        asymplot <- asymmetry_plot(subjectdf,l2e('Foveal'),l2e('Foveal'),colors)

        # title
        if(subjectdf$noncompliant[1]){
            print(paste("Screen size:",round(screen_size(subjectdf),1),"cm")) #testing
            title <- textGrob(paste("ID:",as.character(subjectid),"|","Catch Score:",catch_score(datadf),
                              "Screen size:",round(screen_size(subjectdf)/10,1),"cm",
                              "Viewing distance:",round(view_distance(subjectdf)/10,1),"cm"),
                              gp=gpar(col="red",fontsize=35))
        } else {
            print(paste("Screen size:",round(screen_size(subjectdf),1),"cm")) #testing
            title <- textGrob(paste("ID:",as.character(subjectid),"|","Catch Score:",catch_score(datadf),
                              "Screen size:",round(screen_size(subjectdf)/10,1),"cm",
                              "Viewing distance:",round(view_distance(subjectdf)/10,1),"cm"),
                              gp=gpar(col="blue",fontsize=35))
        }
    
        if(FP){

            # FP Matrix plot
            FPmatrix <- dissimplot(dissimdata(subjectdf, colors, l2e('Foveal'), l2e('Peripheral')),colors)

            # FP CMDS plot
            FPcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(subjectdf,l2e('Foveal'), l2e('Peripheral')))), 
                            abcolors, abcolors, l2e('Foveal'), l2e('Peripheral'))

            # FP nMDS plot
            FPnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(subjectdf,l2e('Foveal'), l2e('Peripheral')))), 
                            abcolors, abcolors, l2e('Foveal'), l2e('Peripheral'))

            # put all the plots together
            g <- arrangeGrob(FFmatrix, FPmatrix, PPmatrix, simplot,
                        FFcmds, FPcmds, PPcmds, reactionplot,
                        FFnmds, FPnmds, PPnmds, rainplot,
                        traceplot, antiplot, differenceplot, 
                        nrow = 4, ncol=4, top = title)

            exp_type <- trial_condition
            ggsave(paste(savestr,"_Subject_Plots/",exp_type,"_ID_",subjectid,"_plot.png",sep=''),
                     g, height = 40, width = 40)
        } else {
            # put all the plots together
            g <- arrangeGrob(FFmatrix, PPmatrix, simplot, traceplot,
                        FFcmds, PPcmds, reactionplot, antiplot,
                        FFnmds, PPnmds, rainplot, 
                        nrow = 3, ncol=4, top = title)

            exp_type <- trial_condition
            ggsave(paste(savestr,"_Subject_Plots/",exp_type,"_ID_",subjectid,"_plot.png",
                     sep=''),g, height = 30, width = 40)
        }
    }
}

In [427]:
if(plotsubjects){
    allsubjectsplot(datadf,savestr)
}

[1] "Processing subject: 162264632"
initial  value 23.903535 
iter   5 value 18.945630
iter   5 value 18.926830
iter   5 value 18.916782
final  value 18.916782 
converged
initial  value 11.972860 
iter   5 value 9.466931
final  value 9.277695 
converged


Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.


[1] "Screen size: 268.1 cm"
initial  value 22.740262 
iter   5 value 19.490939
final  value 18.930836 
converged


`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.


[1] "Processing subject: 235749592"
initial  value 19.692279 
iter   5 value 12.253361
iter  10 value 8.332720
iter  15 value 6.139715
iter  20 value 4.187812
iter  25 value 4.009471
iter  25 value 4.007180
iter  25 value 4.007180
final  value 4.007180 
converged
initial  value 26.186658 
iter   5 value 15.587525
iter  10 value 9.865902
iter  15 value 9.485178
iter  20 value 9.122364
iter  20 value 9.118117
iter  20 value 9.118117
final  value 9.118117 
converged


...

`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.


[1] "Processing subject: 856272856"
initial  value 21.580045 
iter   5 value 15.434249
iter  10 value 14.870485
final  value 14.785546 
converged
initial  value 14.493520 
iter   5 value 11.266563
iter  10 value 10.692085
final  value 10.651713 
converged


Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.


[1] "Screen size: 325.1 cm"


ERROR: Error in cmdscale(distmatrix, k = 2): NA values not allowed in 'd'


#### New mixed trial type version analysis

In [62]:
# matrix to define layout for the mixed trial type plot
mixed_hlay <- rbind (c(1,2,3,4,5,6),
                     c(7,8,9,10,11,12),
                     c(13,14,15,16,17,18))

In [63]:
# names for the x axis
col_plot_list <- c('FF Matrix', 'FF nMDS', 'PP Matrix', 'PP nMDS', 'RT', 'Dissim Freq')

In [264]:
# calculate the catch trial score for a subject
catch_score <- function(datadf){
    datadf <- subset(datadf, trial_type == 'catch')
    datadf$correct <- ifelse(datadf$similarity == datadf$catch_vals, 1, 0) # determine whether they got the catch trials right
    score <- sum(datadf$correct)/nrow(datadf) # get the score
    return(score)
}

In [65]:
# make a plot of the key summary data for a subject - mixed trial type version
subject_mixed_plot <- function(datadf,savestr){
    
    dir.create(paste(savestr,"Mixed_Trial_Type_Plots", sep='_'))
    subjectlist <- sort(unique(datadf$subject)) # obtain a list of all the subjects
    
    
    for (subjectid in subjectlist){ # do for each subject
        
        k = 0 # counter
        plot_list <- list() # list to store the plots in as we're making them
        subjectdf = subset(datadf, subject == subjectid) # select the ID for subject of interest

        for (trial_condition in trial_types){
            type_df = subset(subjectdf, trial_type == trial_condition) # just pick one trial type at a time
            
            # FF Matrix plot
            FFmatrix <- dissimplot(dissimdata(type_df, colors, l2e('Foveal'), l2e('Foveal')),colors)
            k = k + 1
            plot_list[[k]] <- as.grob(FFmatrix)
            
            # FF nMDS plot
            FFnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(type_df,l2e('Foveal'), l2e('Foveal')))), 
                                abcolors, abcolors, l2e('Foveal'), l2e('Foveal'))
            k = k + 1
            plot_list[[k]] <- as.grob(FFnmds)

            # PP Matrix plot
            PPmatrix <- dissimplot(dissimdata(type_df, colors, l2e('Peripheral'), l2e('Peripheral')),colors)
            k = k + 1
            plot_list[[k]] <- as.grob(PPmatrix)

            # PP nMDS plot
            PPnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(type_df,l2e('Peripheral'), l2e('Peripheral')))), 
                                abcolors, abcolors, l2e('Peripheral'), l2e('Peripheral'))
            k = k + 1
            plot_list[[k]] <- as.grob(PPnmds)

            # rt rain plot
            rainplot <- rsplot_raincloud(type_df)
            k = k + 1
            plot_list[[k]] <- as.grob(rainplot)
            # Similarity judgements
            simplot <- simhistplot(subjectdf)
            k = k + 1
            plot_list[[k]] <- as.grob(simplot)

        }

        # save the plots
        yaxis <- textGrob("Trial Type", gp=gpar(fontsize=30), rot=90)
        xaxis <- textGrob("", gp=gpar(fontsize=30))

        title <- textGrob(paste("ID:",subjectid,"|","Catch Score:",catch_score(subjectdf)),gp=gpar(fontsize=35))

        g <- arrangeGrob(grobs = plot_list, ncol = 6,nrow = 3,
                         layout_matrix = mixed_hlay)

        g_rowcol_names <- rbind(tableGrob(t(col_plot_list), theme = ttheme_minimal(), rows = ""),
                    cbind(tableGrob(trial_types, theme = ttheme_minimal()),
                          g, size = "last"), size = "last")


        g_titled <- arrangeGrob(g_rowcol_names, top = title, left = yaxis, bottom = xaxis)
        
        savename <- paste(savestr,"_Mixed_Trial_Type_Plots/",subjectid,"_plot.png",sep='')
        ggsave(savename,g_titled, height = 19, width = 39)
    }
}

In [66]:
subject_mixed_plot(datadf,savestr)

initial  value 1.094606 
final  value 0.000000 
converged
initial  value 5.672467 
final  value 0.000000 
converged
initial  value 0.465013 
final  value 0.000000 
converged
initial  value 12.917472 
iter   5 value 2.830934
iter  10 value 0.520020
iter  15 value 0.119869
iter  20 value 0.025455
final  value 0.008443 
converged
initial  value 7.441465 
iter   5 value 1.277978
iter  10 value 0.384073
iter  15 value 0.084641
final  value 0.006737 
converged
initial  value 3.614121 
final  value 0.000000 
converged
initial  value 34.007888 
iter   5 value 17.550441
iter  10 value 12.364038
iter  15 value 5.457904
iter  20 value 1.123729
iter  25 value 0.477725
iter  30 value 0.066640
final  value 0.003816 
converged
initial  value 23.670976 
iter   5 value 6.298162
iter  10 value 2.789714
iter  15 value 2.105118
final  value 2.099006 
converged
initial  value 15.651304 
iter   5 value 6.566538
iter  10 value 5.543498
final  value 5.506203 
converged
initial  value 12.896237 
iter   5 value

### Summary plots

In [111]:
dir.create(paste(savestr,"Summary_Plots",sep='_'))

In [112]:
# matrix to use to define the plot layout, specified manually for now
hlay <- rbind (c(1,4,7,10),
               c(2,5,8,11),
               c(3,6,9,12))

In [462]:
# correlations summary plot
across_experiment_plot <- function(datadf,savestr,laymatrix,response_type_list,response_names,dependent,clean=FALSE){
    
    dir.create(paste(savestr,"Overall_Summary_Plots",sep='_'))
    
    stimuli_list <- sort(unique(datadf$stimuli_count))
    plot_list <- list()
    
    if(clean){
        datadf <- subset(datadf, noncompliant == 0)
    }
    
    # make all the plots
    k = 0
    for (stimulus in stimuli_list){
        for (response in response_type_list){
            k = k + 1
            if(stimulus == 2){ # 2 stimuli doesn't have any cueing, so it gets an exception
                tempdf <- subset(datadf, stimuli_count == 2)
            } else {
                tempdf <- subset(datadf, stimuli_count == stimulus & response_type == response)
            }
            
            # make the specific plot
            if(dependent=='correlation'){
                fvp <- fvp_list(tempdf)
                plot <- corr_plot(fvp,'Within-subject FvP','Within-subject FvP')
                title <- textGrob("CORRELATION", gp=gpar(fontsize=35))
            } else if (dependent=='RT'){
                plot <- rsplot_summary(tempdf)
                title <- textGrob("REACTION TIME", gp=gpar(fontsize=35))
            } else if (dependent=='dissimilarity'){
                plot <- simhistplot_summary(tempdf)
                title <- textGrob("DISSIMILARITY", gp=gpar(fontsize=35))
            } else if (dependent=='trace'){
                plot <- trace_plot(tempdf)
                title <- textGrob("TRACE", gp=gpar(fontsize=35))
            } else if (dependent=='antitrace'){
                plot <- trace_plot(tempdf,'antitrace')
                title <- textGrob("ANTITRACE", gp=gpar(fontsize=35))
            } else if (dependent=='difference'){
                plot <- trace_plot(tempdf,'difference')
                title <- textGrob("ANTITRACE - TRACE", gp=gpar(fontsize=35))
            } else if (dependent=='asymmetry_matrix'){
                plot <- asymmetry_plot(tempdf,l2e('Foveal'),l2e('Foveal'),colors)
                title <- textGrob("SIMILARITY ASYMMETRY MATRIX", gp=gpar(fontsize=35))
            } else if (dependent == 'trace_by_trial'){
                plot <- trace_by_trial_plot(tempdf,'boxplot')
                title <- textGrob("Trace by Trial", gp=gpar(fontsize=35))
            }
            
            
            plot <- plot + xlab(paste(stimulus,response,sep=' ')) #  add a label
            if(stimulus == 2 && k != 1){ # don't plot after-cueing for 2 cue stimuli
                plot <- ggplot() + geom_blank() + theme_void()
            } else if (dependent == 'asymmetry_matrix' && response != 'after_1'){
                plot <- ggplot() + geom_blank() + theme_void()
            }
            plot_list[[k]] <- as.grob(plot) # add it to the plot_list
        }
        
    }
    
    # save the plots
    yaxis <- textGrob("Cueing Type", gp=gpar(fontsize=30), rot=90)
    xaxis <- textGrob("Stimuli", gp=gpar(fontsize=30))
    
    g <- arrangeGrob(grobs = plot_list, ncol = length(stimuli_list),nrow = length(response_type_list),
                     layout_matrix = hlay)
    
    g_rowcol_names <- rbind(tableGrob(t(stimuli_list), theme = ttheme_minimal(), rows = ""),
                cbind(tableGrob(response_names, theme = ttheme_minimal()),
                      g, size = "last"), size = "last")
    
    g_titled <- arrangeGrob(g_rowcol_names, top = title, left = yaxis, bottom = xaxis)
    
    if(clean){
        savename <- paste(savestr,"_Overall_Summary_Plots/",dependent,"_CLEAN","_plot.png",sep='')
        ggsave(savename,g_titled, height = 20, width = 30)
    }else{
        savename <- paste(savestr,"_Overall_Summary_Plots/",dependent,"_plot.png",sep='')
        ggsave(savename,g_titled, height = 20, width = 30)
    }
    
    return(g_titled)
}

In [463]:
across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'trace_by_trial')
across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'trace_by_trial',TRUE)

TableGrob (3 x 2) "arrange": 4 grobs
  z     cells    name                   grob
1 1 (2-2,2-2) arrange     gtable[rowhead-fg]
2 2 (1-1,2-2) arrange text[GRID.text.323514]
3 3 (3-3,2-2) arrange text[GRID.text.323785]
4 4 (1-3,1-1) arrange text[GRID.text.323784]

TableGrob (3 x 2) "arrange": 4 grobs
  z     cells    name                   grob
1 1 (2-2,2-2) arrange     gtable[rowhead-fg]
2 2 (1-1,2-2) arrange text[GRID.text.326306]
3 3 (3-3,2-2) arrange text[GRID.text.326567]
4 4 (1-3,1-1) arrange text[GRID.text.326566]

#### Across all experiments

In [202]:
across_all <- function(datadf,savestr,laymatrix,response_type_list,response_names){
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'RT')
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'dissimilarity')
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'asymmetry_matrix')
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'correlation')
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'trace')
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'antitrace')
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'difference')
    
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'RT',TRUE)
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'dissimilarity',TRUE)
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'asymmetry_matrix',TRUE)
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'correlation',TRUE)
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'trace',TRUE)
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'antitrace',TRUE)
    across_experiment_plot(datadf,savestr,hlay,response_type_list,response_names,'difference',TRUE)
}

In [203]:
if(across){
    across_all(datadf,savestr,hlay,response_type_list,response_names)
}

`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` usin

ERROR: Error in across_all(datadf, savestr, hlay, response_type_list, response_names, : unused argument (TRUE)


#### Individual experiment summaries

In [188]:
# generate a summary plot of all the collected data from all the subjects in the experiment
# NEED TO FINISH THIS
expsummaryplot <- function(datadf,savestr,clean=FALSE){
    
    dir.create(paste(savestr,"Exp_Summary_Plots/",sep='_'))
    
    if(clean){
        datadf <- subset(datadf, noncompliant == 0)
    }
    
    stimuli_list <- sort(unique(datadf$stimuli_count))
    
    # make all the plots
    k = 0
    for (stimulus in stimuli_list){
        for (response in response_type_list){
            
            k = k + 1
            if(stimulus == 2){ # 2 stimuli doesn't have any cueing, so it gets an exception
                tempdf <- subset(datadf, stimuli_count == 2)
            } else {
                tempdf <- subset(datadf, stimuli_count == stimulus & response_type == response)
            }
            
            if(response != 'after_1'){
                FP = TRUE
            } else {
                FP = FALSE
            }
            
            # make the specific summary plot
           
            # MATRICES
            everyonedata <- aggregate_df(tempdf)
            
            stim_time <- everyonedata$stim_display_time[1]
            gap <- everyonedata$stim_cue_gap[1]
            cue_time <- everyonedata$cue_display_time[1]
            
            
            # FF Matrix plot
            FFmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Foveal'), l2e('Foveal')),colors)

            # FP Matrix plot
            FPmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Foveal'), l2e('Peripheral')),colors)

            # PP Matrix plot
            PPmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Peripheral'), l2e('Peripheral')),colors)

            # CMDS
            # FF CMDS plot
            FFcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(everyonedata,l2e('Foveal'), l2e('Foveal')))), 
                                abcolors, abcolors, l2e('Foveal'), l2e('Foveal'))

            FPcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(everyonedata,l2e('Foveal'), l2e('Peripheral')))), 
                                abcolors, abcolors, l2e('Foveal'), l2e('Peripheral'))

            # FF CMDS plot
            PPcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(everyonedata,l2e('Peripheral'), l2e('Peripheral')))), 
                                abcolors, abcolors, l2e('Peripheral'), l2e('Peripheral'))
            # NMDS
            # FF nMDS plot
            FFnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(everyonedata,l2e('Foveal'), l2e('Foveal')))), 
                                abcolors, abcolors, l2e('Foveal'), l2e('Foveal'))

            # FP nMDS plot
            FPnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(everyonedata,l2e('Foveal'), l2e('Peripheral')))), 
                                abcolors, abcolors, l2e('Foveal'), l2e('Peripheral'))

            # PP nMDS plot
            PPnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(everyonedata,l2e('Peripheral'), l2e('Peripheral')))), 
                                abcolors, abcolors, l2e('Peripheral'), l2e('Peripheral'))

            # AUXILIARY plots
            # Similarity judgements
            simplot <- simhistplot_summary(tempdf)
            # Reaction times v1
            reactionplot <- rsplot_summary(tempdf)
            # summary plot
            raincloudplot <- rsplot_raincloud(tempdf,'log')

            # within_between_plots
            fvp <- fvp_list(tempdf)
            i_j <- i_j_list(tempdf)

            within_plot <- corr_plot(fvp,'Within-subject FvP','Within-subject FvP')
            between_plot <- corr_plot(i_j,'Between-subject FvF','Between-subject FvF')
            #within_plot_fisher <- corr_plot(FisherZ(fvp),'Within-subject FvP','Within-subject FvP',TRUE)
            #between_plot_fisher <- corr_plot(FisherZ(i_j),'Between-subject FvF','Between-subject FvF',TRUE)

            # MORE PLOTS
            # trace plots
            traceplot <- trace_plot(tempdf)
            antiplot <- trace_plot(tempdf,'antitrace')
            differenceplot <- trace_plot(tempdf,'difference')
            
            N <- length(unique(tempdf$subject)) # get number of subjects
            
            # save the plots
            if(clean){
                title <- textGrob(paste(stimulus,response,
                                    "stim_duration",stim_time,"gap",gap,"cue_duration",cue_time,
                                    N,"subjects","CLEAN",
                                    sep="_"), gp=gpar(fontsize=35))
            }else{
                title <- textGrob(paste(stimulus,response,
                                    "stim_duration",stim_time,"gap",gap,"cue_duration",cue_time,
                                    N,"subjects",
                                    sep="_"), gp=gpar(fontsize=35))
            }
            
            # put it all together
            if(FP){
                
                g <- arrangeGrob(FFmatrix, FPmatrix, PPmatrix, within_plot, between_plot,
                                FFcmds,FPcmds, PPcmds, traceplot, antiplot,
                                FFnmds,FPnmds, PPnmds, raincloudplot, differenceplot,
                                simplot, reactionplot, 
                                nrow = 4, ncol=5, top = title)
            } else {
                
                # asymmetry plot
                asymplot <- asymmetry_plot(tempdf,l2e('Foveal'),l2e('Foveal'),colors)
                
                g <- arrangeGrob(FFmatrix,  PPmatrix, within_plot, between_plot, 
                                FFcmds, PPcmds, traceplot,  antiplot,
                                FFnmds, PPnmds, differenceplot, asymplot,
                                simplot, raincloudplot, reactionplot,
                                nrow = 4, ncol=4, top = title)

                
            }
                # save it
            if(clean){
                savename <- paste(savestr,"_Exp_Summary_Plots/",stimulus,"_",response,
                         "_Summary_",N,"_subjects","_CLEAN","_plot.png",sep='')
            }else {
                savename <- paste(savestr,"_Exp_Summary_Plots/",stimulus,"_",response,
                         "_Summary_",N,"_subjects","_plot.png",sep='')
            }
                
                print(savename)
                ggsave(savename,g, height = 38, width = 48)
        }
            
    }
        
}   

### Across the entire population 

In [213]:
# entire population plot
population_plot <- function(datadf,savestr,clean=FALSE){
    
    dir.create(paste(savestr,"Population_Summary_Plots/",sep='_'))
    
    if(clean){
        datadf <- subset(datadf, noncompliant == 0)
    }
    
    # MATRICES
    everyonedata <- aggregate_df(datadf)

    # FF Matrix plot
    FFmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Foveal'), l2e('Foveal')),colors)

    # FP Matrix plot
    FPmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Foveal'), l2e('Peripheral')),colors)

    # PP Matrix plot
    PPmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Peripheral'), l2e('Peripheral')),colors)

    # CMDS
    # FF CMDS plot
    FFcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(everyonedata,l2e('Foveal'), l2e('Foveal')))), 
                        abcolors, abcolors, l2e('Foveal'), l2e('Foveal'))

    FPcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(everyonedata,l2e('Foveal'), l2e('Peripheral')))), 
                        abcolors, abcolors, l2e('Foveal'), l2e('Peripheral'))

    # FF CMDS plot
    PPcmds <-  cmdsplot(cmdsfit(mat2dist(df2mat(everyonedata,l2e('Peripheral'), l2e('Peripheral')))), 
                        abcolors, abcolors, l2e('Peripheral'), l2e('Peripheral'))
    # NMDS
    # FF nMDS plot
    FFnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(everyonedata,l2e('Foveal'), l2e('Foveal')))), 
                        abcolors, abcolors, l2e('Foveal'), l2e('Foveal'))

    # FP nMDS plot
    FPnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(everyonedata,l2e('Foveal'), l2e('Peripheral')))), 
                        abcolors, abcolors, l2e('Foveal'), l2e('Peripheral'))

    # PP nMDS plot
    PPnmds <-  nmdsplot(nmdsfit(mat2dist(df2mat(everyonedata,l2e('Peripheral'), l2e('Peripheral')))), 
                        abcolors, abcolors, l2e('Peripheral'), l2e('Peripheral'))

    # AUXILIARY plots
    # Similarity judgements
    simplot <- simhistplot_summary(datadf)
    # Reaction times v1
    reactionplot <- rsplot_summary(datadf)
    # summary plot
    raincloudplot <- rsplot_raincloud(datadf,'log')

    # within_between_plots
    fvp <- fvp_list(datadf)
    i_j <- i_j_list(datadf)

    within_plot <- corr_plot(fvp,'Within-subject FvP','Within-subject FvP')
    between_plot <- corr_plot(i_j,'Between-subject FvF','Between-subject FvF')
    #within_plot_fisher <- corr_plot(FisherZ(fvp),'Within-subject FvP','Within-subject FvP',TRUE)
    #between_plot_fisher <- corr_plot(FisherZ(i_j),'Between-subject FvF','Between-subject FvF',TRUE)

    # MORE PLOTS
    # trace plots
    traceplot <- trace_plot(datadf)
    antiplot <- trace_plot(datadf,'antitrace')
    differenceplot <- trace_plot(datadf,'difference')

    N <- length(unique(datadf$subject)) # get number of subjects

    # save the plots
    if(clean){
        title <- textGrob(paste("Population",
                            N,"subjects","CLEAN",
                            sep="_"), gp=gpar(fontsize=35))
    }else{
        title <- textGrob(paste("Population",
                            N,"subjects",
                            sep="_"), gp=gpar(fontsize=35))
    }
    
    
    g <- arrangeGrob(FFmatrix, FPmatrix, PPmatrix, within_plot, between_plot,
                                FFcmds,FPcmds, PPcmds, traceplot, antiplot,
                                FFnmds,FPnmds, PPnmds, raincloudplot, differenceplot,
                                simplot, reactionplot, 
                                nrow = 4, ncol=5, top = title)
    
     # save it
    if(clean){
        savename <- paste(savestr,"_Exp_Summary_Plots/",
                 "_Summary_",N,"_subjects","_CLEAN","_plot.png",sep='')
    }else {
        savename <- paste(savestr,"_Exp_Summary_Plots/",
                 "_Summary_",N,"_subjects","_plot.png",sep='')
    }

        print(savename)
        ggsave(savename,g, height = 38, width = 48)
}

In [2]:
test <- dissimplot(dissimdata(everyonedata, colors, l2e('Foveal'), l2e('Foveal')),colors)

test

ERROR: Error in dissimplot(dissimdata(everyonedata, colors, l2e("Foveal"), l2e("Foveal")), : could not find function "dissimplot"


In [214]:
if(plotexpsummary){
    expsummaryplot(datadf,savestr)
    expsummaryplot(datadf,savestr,TRUE)
}

In [215]:
if(population){
    population_plot(datadf,savestr)
    population_plot(datadf,savestr,TRUE)
}

initial  value 10.110642 
iter   5 value 4.776044
iter  10 value 4.309526
final  value 4.276221 
converged
initial  value 9.398094 
iter   5 value 4.759062
iter  10 value 3.994180
final  value 3.981702 
converged
initial  value 5.047839 
iter   5 value 1.323556
iter  10 value 0.944285
iter  15 value 0.841798
iter  20 value 0.821025
final  value 0.817766 
converged


`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.


[1] "data/alldata_Exp_Summary_Plots/_Summary_128_subjects_plot.png"
initial  value 3.365119 
iter   5 value 1.843522
iter  10 value 1.615815
iter  15 value 1.571471
final  value 1.567521 
converged
initial  value 5.607216 
iter   5 value 2.817553
iter  10 value 2.443810
final  value 2.402800 
converged
initial  value 4.294241 
iter   5 value 2.475724
iter  10 value 2.280757
final  value 2.271904 
converged


`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.


[1] "data/alldata_Exp_Summary_Plots/_Summary_37_subjects_CLEAN_plot.png"


In [ ]:
# position plot (negative control)
positionplot <- function(datadf){
    # MATRICES
    everyonedata <- aggregate_df(datadf)
    # FF Matrix plot
    FFmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Foveal'), l2e('Foveal')),colors, 'position')
    # FP Matrix plot
    FPmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Foveal'), l2e('Peripheral')),colors, 'position')
    # PP Matrix plot
    PPmatrix <- dissimplot(dissimdata(everyonedata, colors, l2e('Peripheral'), l2e('Peripheral')),colors, 'position')
    
    g <- arrangeGrob(FFmatrix, FPmatrix, PPmatrix,
                    nrow = 1, ncol= 3)
     # save it
    ggsave(paste(savestr,"position negative control","plot.png",sep='_'),g, height = 10, width = 30)
}